In [247]:
import pandas as pd
import numpy as np
import datetime
from time import strftime

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [248]:
week_key = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

In [249]:
df = pd.read_csv("/home/cslab3pc25/Downloads/data.csv")

In [250]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [251]:
df['PatientId'] = df['PatientId'].astype('int64')
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.date.astype('datetime64[ns]')
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.date.astype('datetime64[ns]')
df = df.rename(columns={'Hipertension': 'Hypertension', 'Handcap': 'Handicap', 'SMS_received': 'SMSReceived', 'No-show': 'NoShow'})

In [252]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null int64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null datetime64[ns]
AppointmentDay    110527 non-null datetime64[ns]
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hypertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handicap          110527 non-null int64
SMSReceived       110527 non-null int64
NoShow            110527 non-null object
dtypes: datetime64[ns](2), int64(9), object(3)
memory usage: 11.8+ MB


In [253]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow
0,29872499824296,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,558997776694438,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4262962299951,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,867951213174,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8841186448183,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [254]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hypertension',
       'Diabetes', 'Alcoholism', 'Handicap', 'SMSReceived', 'NoShow'],
      dtype='object')

In [255]:
df.drop(['PatientId', 'AppointmentID'], axis=1, inplace=True)


In [256]:
df['Scholarship'] = df['Scholarship'].astype('object')
df['Hypertension'] = df['Hypertension'].astype('object')
df['Diabetes'] = df['Diabetes'].astype('object')
df['Alcoholism'] = df['Alcoholism'].astype('object')
df['Handicap'] = df['Handicap'].astype('object')
df['SMSReceived'] = df['SMSReceived'].astype('object')
df['Neighbourhood']=df['Neighbourhood'].astype('object')

In [257]:
np.sort(df.Age.unique())

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 102, 115])

In [258]:
df=df[df.Age>=0]

In [259]:
np.sort(df.Age.unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 102, 115])

In [260]:
df[(df.Age <= 0) & ((df.Hypertension.astype(int) == 1) | (df.Diabetes.astype(int) == 1) | (df.Alcoholism.astype(int) == 1))]

,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow


In [261]:
print("Unique Values in `ScheduledDay` => {}".format(np.sort(df.ScheduledDay.dt.strftime('%d-%m-%y').unique())))

Unique Values in `ScheduledDay` => ['01-02-16' '01-03-16' '01-04-16' '01-06-16' '02-02-16' '02-03-16'
 '02-05-16' '02-06-16' '03-02-16' '03-03-16' '03-05-16' '03-06-16'
 '03-12-15' '04-01-16' '04-02-16' '04-03-16' '04-05-16' '04-06-16'
 '05-01-16' '05-02-16' '05-03-16' '05-04-16' '05-05-16' '06-04-16'
 '06-05-16' '06-06-16' '07-01-16' '07-03-16' '07-04-16' '07-05-16'
 '07-06-16' '07-12-15' '08-03-16' '08-04-16' '08-06-16' '08-12-15'
 '09-03-16' '09-04-16' '09-05-16' '10-03-16' '10-05-16' '10-11-15'
 '11-01-16' '11-02-16' '11-03-16' '11-04-16' '11-05-16' '12-02-16'
 '12-04-16' '12-05-16' '13-01-16' '13-04-16' '13-05-16' '14-01-16'
 '14-03-16' '14-04-16' '14-05-16' '14-12-15' '15-02-16' '15-03-16'
 '15-04-16' '15-12-15' '16-02-16' '16-03-16' '16-04-16' '16-05-16'
 '17-02-16' '17-03-16' '17-05-16' '18-02-16' '18-03-16' '18-04-16'
 '18-05-16' '19-01-16' '19-02-16' '19-03-16' '19-04-16' '19-05-16'
 '20-01-16' '20-04-16' '20-05-16' '21-01-16' '21-03-16' '22-01-16'
 '22-02-16' '22-03-16' '23-

In [262]:
print("Unique Values in `AppointmentDay` => {}".format(np.sort(df.AppointmentDay.dt.strftime('%d-%m-%y').unique())))

Unique Values in `AppointmentDay` => ['01-06-16' '02-05-16' '02-06-16' '03-05-16' '03-06-16' '04-05-16'
 '05-05-16' '06-05-16' '06-06-16' '07-06-16' '08-06-16' '09-05-16'
 '10-05-16' '11-05-16' '12-05-16' '13-05-16' '14-05-16' '16-05-16'
 '17-05-16' '18-05-16' '19-05-16' '20-05-16' '24-05-16' '25-05-16'
 '29-04-16' '30-05-16' '31-05-16']


In [263]:
df['ScheduledDay_DOW'] = df['ScheduledDay'].dt.weekday_name
df['AppointmentDay_DOW'] = df['AppointmentDay'].dt.weekday_name

In [264]:
df['AppointmentDay'] = np.where((df['AppointmentDay'] - df['ScheduledDay']).dt.days < 0, df['ScheduledDay'], df['AppointmentDay'])
df['Waiting_Time_days'] = df['AppointmentDay'] - df['ScheduledDay']
df['Waiting_Time_days'] = df['Waiting_Time_days'].dt.days

In [265]:
df.sample(n=10)

,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSReceived,NoShow,ScheduledDay_DOW,AppointmentDay_DOW,Waiting_Time_days
51530,F,2016-04-25,2016-05-11,40,CENTRO,0,0,0,0,0,1,No,Monday,Wednesday,16
37865,M,2016-05-12,2016-05-12,25,DE LOURDES,0,0,0,0,0,0,No,Thursday,Thursday,0
44556,F,2016-05-12,2016-05-12,12,GOIABEIRAS,0,0,0,0,0,0,No,Thursday,Thursday,0
61781,F,2016-05-16,2016-05-24,85,PRAIA DO SUÁ,0,1,1,0,0,1,Yes,Monday,Tuesday,8
106637,F,2016-06-02,2016-06-02,25,BONFIM,0,0,0,0,0,0,No,Thursday,Thursday,0
18971,F,2016-05-16,2016-05-16,22,FORTE SÃO JOÃO,0,0,0,0,0,0,No,Monday,Monday,0
881,M,2016-02-24,2016-04-29,47,SANTOS DUMONT,0,1,0,0,0,1,No,Wednesday,Friday,65
78478,F,2016-05-17,2016-05-17,50,BONFIM,0,1,1,0,0,0,No,Tuesday,Tuesday,0
71185,M,2016-03-10,2016-05-17,67,FONTE GRANDE,0,0,0,0,0,0,Yes,Thursday,Tuesday,68
94212,F,2016-05-16,2016-06-02,60,CARATOÍRA,0,1,0,0,0,1,No,Monday,Thursday,17


In [266]:
print("NoShow and Show Count of Patients\n")
print(df.groupby(['NoShow']).size())

print("\nNoShow and Show '%' of Patients\n")
show = df.groupby(['NoShow']).size()[0]/(df.groupby(['NoShow']).size()[0]+df.groupby(['NoShow']).size()[1])
print("Percent of Patients who `Showed Up` => {:.2f}%".format(show*100))
noshow = df.groupby(['NoShow']).size()[1]/(df.groupby(['NoShow']).size()[0]+df.groupby(['NoShow']).size()[1])
print("Percent of Patients who Did `Not Showed Up` => {:.2f}%".format(noshow*100))

NoShow and Show Count of Patients

NoShow
No     88207
Yes    22319
dtype: int64

NoShow and Show '%' of Patients

Percent of Patients who `Showed Up` => 79.81%
Percent of Patients who Did `Not Showed Up` => 20.19%


In [274]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
le = LabelEncoder()
df['Neighbourhood'] = le.fit_transform(df['Neighbourhood'])

le = LabelEncoder()
df['ScheduledDay_DOW'] = le.fit_transform(df['ScheduledDay_DOW'])

le = LabelEncoder()
df['AppointmentDay_DOW'] = le.fit_transform(df['AppointmentDay_DOW'])
print("LabelEncoder Completed")

le = LabelEncoder()
df['NoShow'] = le.fit_transform(df['NoShow'])


LabelEncoder Completed


In [275]:
df['ScheduledDay_Y'] = df['ScheduledDay'].dt.year
df['ScheduledDay_M'] = df['ScheduledDay'].dt.month
df['ScheduledDay_D'] = df['ScheduledDay'].dt.day
df.drop(['ScheduledDay'], axis=1, inplace=True)

df['AppointmentDay_Y'] = df['AppointmentDay'].dt.year
df['AppointmentDay_M'] = df['AppointmentDay'].dt.month
df['AppointmentDay_D'] = df['AppointmentDay'].dt.day
df.drop(['AppointmentDay'], axis=1, inplace=True)

In [276]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 110526 entries, 0 to 110526
Data columns (total 19 columns):
Gender                110526 non-null int64
Age                   110526 non-null int64
Neighbourhood         110526 non-null int64
Scholarship           110526 non-null object
Hypertension          110526 non-null object
Diabetes              110526 non-null object
Alcoholism            110526 non-null object
Handicap              110526 non-null object
SMSReceived           110526 non-null object
NoShow                110526 non-null int64
ScheduledDay_DOW      110526 non-null int64
AppointmentDay_DOW    110526 non-null int64
Waiting_Time_days     110526 non-null int64
ScheduledDay_Y        110526 non-null int64
ScheduledDay_M        110526 non-null int64
ScheduledDay_D        110526 non-null int64
AppointmentDay_Y      110526 non-null int64
AppointmentDay_M      110526 non-null int64
AppointmentDay_D      110526 non-null int64
dtypes: int64(13), object(6)
memory usage: 16.9

In [277]:
X = df.drop(['NoShow'], axis=1)
y = df['NoShow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [278]:
print("Feature Importance:\n")
for name, importance in zip(X.columns, np.sort(dt_clf.feature_importances_)[::-1]):
    print("{} -- {:.2f}".format(name, importance))

Feature Importance:

Gender -- 0.23
Age -- 0.23
Neighbourhood -- 0.17
Scholarship -- 0.08
Hypertension -- 0.07
Diabetes -- 0.05
Alcoholism -- 0.05
Handicap -- 0.04
SMSReceived -- 0.02
ScheduledDay_DOW -- 0.02
AppointmentDay_DOW -- 0.01
Waiting_Time_days -- 0.01
ScheduledDay_Y -- 0.01
ScheduledDay_M -- 0.01
ScheduledDay_D -- 0.01
AppointmentDay_Y -- 0.01
AppointmentDay_M -- 0.00
AppointmentDay_D -- 0.00


In [279]:
dt_clf.score(X_test, y_test)

0.723419422054066